In [1]:
import librosa
import matplotlib.pyplot as plt
import librosa.display
import os
import numpy as np
from tensorflow.keras import layers
from IPython.display import Audio
import wave
import pandas as pd
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

2024-02-29 20:56:55.468949: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 20:56:55.468992: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 20:56:55.470655: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-29 20:56:55.479587: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-29 20:56:56.540872: W tensorflow/compiler/tf2

In [2]:
df = pd.read_csv("../data_processing/sep28k-mfcc.csv")

In [3]:
df = df[df['NaturalPause'] == 0]
df = df[df['Interjection'] == 0]
df = df[df['WordRep'] == 0]
df = df[df['SoundRep'] == 0]
df = df[df['Block'] == 0]
df.head()

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,...,29,30,31,32,33,34,35,36,37,38
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,...,-3.249218,-3.218731,0.360520,-0.115838,1.723229,-1.460772,-1.430723,-1.429669,-0.178196,-3.046811
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,...,-2.084866,0.856960,-0.589489,-0.935007,0.202947,-2.603561,-1.701218,-2.591237,0.655423,0.828504
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,...,-2.057481,-3.225782,-1.561315,-3.723942,-2.249673,-3.513439,-2.097951,-1.940040,0.020210,-0.556461
5,HeStutters,0,5,36435040,36483040,0,0,1,0,0,...,-2.220595,-4.731575,-4.027788,-4.388147,-3.380947,-5.370929,-2.888416,-4.782809,-4.708760,-6.034374
6,HeStutters,0,6,37251200,37299200,0,0,0,0,0,...,-4.985494,-6.181957,-2.396787,-6.544606,-2.319257,-3.688410,-0.221778,-1.690031,0.664374,-0.303436


In [4]:
df = df.reset_index()
df.head()

,index,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,...,29,30,31,32,33,34,35,36,37,38
0,2,HeStutters,0,2,34809760,34857760,0,0,0,0,...,-3.249218,-3.218731,0.360520,-0.115838,1.723229,-1.460772,-1.430723,-1.429669,-0.178196,-3.046811
1,3,HeStutters,0,3,35200640,35248640,0,0,1,0,...,-2.084866,0.856960,-0.589489,-0.935007,0.202947,-2.603561,-1.701218,-2.591237,0.655423,0.828504
2,4,HeStutters,0,4,35721920,35769920,0,0,0,0,...,-2.057481,-3.225782,-1.561315,-3.723942,-2.249673,-3.513439,-2.097951,-1.940040,0.020210,-0.556461
3,5,HeStutters,0,5,36435040,36483040,0,0,1,0,...,-2.220595,-4.731575,-4.027788,-4.388147,-3.380947,-5.370929,-2.888416,-4.782809,-4.708760,-6.034374
4,6,HeStutters,0,6,37251200,37299200,0,0,0,0,...,-4.985494,-6.181957,-2.396787,-6.544606,-2.319257,-3.688410,-0.221778,-1.690031,0.664374,-0.303436


In [5]:
df = df.drop(columns=['index'])
df.head()

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,...,29,30,31,32,33,34,35,36,37,38
0,HeStutters,0,2,34809760,34857760,0,0,0,0,0,...,-3.249218,-3.218731,0.360520,-0.115838,1.723229,-1.460772,-1.430723,-1.429669,-0.178196,-3.046811
1,HeStutters,0,3,35200640,35248640,0,0,1,0,0,...,-2.084866,0.856960,-0.589489,-0.935007,0.202947,-2.603561,-1.701218,-2.591237,0.655423,0.828504
2,HeStutters,0,4,35721920,35769920,0,0,0,0,0,...,-2.057481,-3.225782,-1.561315,-3.723942,-2.249673,-3.513439,-2.097951,-1.940040,0.020210,-0.556461
3,HeStutters,0,5,36435040,36483040,0,0,1,0,0,...,-2.220595,-4.731575,-4.027788,-4.388147,-3.380947,-5.370929,-2.888416,-4.782809,-4.708760,-6.034374
4,HeStutters,0,6,37251200,37299200,0,0,0,0,0,...,-4.985494,-6.181957,-2.396787,-6.544606,-2.319257,-3.688410,-0.221778,-1.690031,0.664374,-0.303436


In [6]:
df.to_csv("prolongation.csv",index=False)

In [7]:
import os

def list_files(directory):
    return [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

# Replace 'directory_path' with the path to your directory
directory_path = '/home/alien/Git/DATA/mel_spects_prolongation'
# names_list = list_files(directory_path)
names_list = pd.read_csv("prolongation.csv")['Name'].values.tolist()
# full_names_list = ["/home/alien/Git/DATA/mfcc_images/" + img + ".jpg" for img in names_list]
full_names_list = []
for img in names_list:
    corresponding_sound = df.loc[df['Name'] == img, 'Prolongation'].values[0]
    if corresponding_sound == 0:
        full_names_list.append("/home/alien/Git/DATA/mel_spects_prolongation/" + img + "_fluent.jpg")
    if corresponding_sound >= 1:
        full_names_list.append("/home/alien/Git/DATA/mel_spects_prolongation/" + img + "_stutter.jpg")

print(full_names_list[-2])

/home/alien/Git/DATA/mel_spects_prolongation/WomenWhoStutter_109_33_fluent.jpg


In [8]:
def load_all(imagefile_list):
    data = []
    labels = []

    for imagefile in imagefile_list:
        print(imagefile)
        image = cv2.imread(imagefile)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))
        data.append(image)

        if "fluent" in imagefile:
            labels.append(0)
        else:
            labels.append(1)

    labels = np.array(labels)
    data = np.array(data)

    return data, labels

In [9]:
X, y = load_all(full_names_list)

/home/alien/Git/DATA/mel_spects_prolongation/HeStutters_0_2_fluent.jpg
/home/alien/Git/DATA/mel_spects_prolongation/HeStutters_0_3_stutter.jpg
/home/alien/Git/DATA/mel_spects_prolongation/HeStutters_0_4_fluent.jpg
/home/alien/Git/DATA/mel_spects_prolongation/HeStutters_0_5_stutter.jpg
/home/alien/Git/DATA/mel_spects_prolongation/HeStutters_0_6_fluent.jpg
/home/alien/Git/DATA/mel_spects_prolongation/HeStutters_0_9_fluent.jpg
/home/alien/Git/DATA/mel_spects_prolongation/HeStutters_0_11_fluent.jpg
/home/alien/Git/DATA/mel_spects_prolongation/HeStutters_0_12_fluent.jpg
/home/alien/Git/DATA/mel_spects_prolongation/HeStutters_0_15_fluent.jpg
/home/alien/Git/DATA/mel_spects_prolongation/HeStutters_0_16_fluent.jpg
/home/alien/Git/DATA/mel_spects_prolongation/HeStutters_0_19_stutter.jpg
/home/alien/Git/DATA/mel_spects_prolongation/HeStutters_0_20_fluent.jpg
/home/alien/Git/DATA/mel_spects_prolongation/HeStutters_0_22_fluent.jpg
/home/alien/Git/DATA/mel_spects_prolongation/HeStutters_0_24_fluent

In [10]:
print(X[0])
print(y[0])

[[[229  87  91]
  [203  68  83]
  [206  69  98]
  ...
  [ 10   0  30]
  [  5   2  22]
  [  1   5   7]]

 [[252 137 103]
  [249 134 108]
  [251 125 117]
  ...
  [ 45  22  80]
  [ 37  24  65]
  [  2   4  12]]

 [[241 144  84]
  [243 145  92]
  [245 131  93]
  ...
  [ 44  13  96]
  [ 40  21  79]
  [  5   1  19]]

 ...

 [[206  65 117]
  [201  62 108]
  [204  61 118]
  ...
  [ 52  16 106]
  [ 55  21 103]
  [ 22   4  63]]

 [[ 86  28  44]
  [ 93  33  48]
  [ 93  30  51]
  ...
  [ 22   7  48]
  [ 24  10  49]
  [ 10   3  32]]

 [[  0   0   2]
  [  0   0   2]
  [  0   0   2]
  ...
  [  0   0   2]
  [  0   0   2]
  [  0   0   2]]]
0


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.2, random_state=12)

In [12]:
print("x_train shape:", X_train.shape)
print("x_test shape:", X_test.shape)
print('y_train shape:', y_train.shape)
print("y_test shape:", y_test.shape)

x_train shape: (2003, 224, 224, 3)
x_test shape: (501, 224, 224, 3)
y_train shape: (2003,)
y_test shape: (501,)


In [13]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

2003 501 2003 501


In [14]:
from collections import Counter
print(Counter(y_train))
print(Counter(y_test))

Counter({0: 1354, 1: 649})
Counter({0: 344, 1: 157})


In [15]:
print(X_train)

[[[[  0   0   4]
   [  0   0   4]
   [  0   0   4]
   ...
   [ 43  15  51]
   [102  30 107]
   [117  25 131]]

  [[  0   0   2]
   [  0   0   2]
   [  0   0   2]
   ...
   [ 50  15  68]
   [100  29 114]
   [110  23 134]]

  [[  0   0   2]
   [  0   0   2]
   [  0   0   2]
   ...
   [ 55  13  83]
   [ 99  28 122]
   [108  27 138]]

  ...

  [[ 67  21 115]
   [ 53  12  97]
   [ 47  13  87]
   ...
   [ 32  20  74]
   [ 28  10  73]
   [ 72  21 117]]

  [[ 29   9  54]
   [ 24   6  46]
   [ 21   7  43]
   ...
   [  9   6  29]
   [ 12   8  34]
   [ 32  11  51]]

  [[  0   0   2]
   [  0   0   2]
   [  0   0   2]
   ...
   [  0   0   2]
   [  0   0   2]
   [  0   0   2]]]


 [[[ 28  13  66]
   [ 15   6  48]
   [  3   2  25]
   ...
   [  1   1   1]
   [  1   1   0]
   [  1   1   0]]

  [[ 41  21  88]
   [ 27  15  70]
   [ 14   9  47]
   ...
   [  0   0   1]
   [  0   0   0]
   [  0   0   0]]

  [[ 45  17  99]
   [ 30  11  81]
   [ 15   6  59]
   ...
   [  2   1   4]
   [  0   0   2]
   [  0   0

In [16]:
from PIL import Image
IMAGE_DIR = "/home/alien/Git/DATA/mel_spects_prolongation/"

def get_image_dimensions(image_path):
    with Image.open(image_path) as img:
        width, height = img.size
    return width, height

image_path = '/home/alien/Git/DATA/mel_spects_prolongation/HeStutters_0_0_fluent.jpg'  # Change this to the path of your image file
width, height = get_image_dimensions(image_path)
print("Image width:", width)
print("Image height:", height)

Image width: 640
Image height: 435


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, MultiHeadAttention
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout, Flatten, Dense, Input, AveragePooling2D, Attention, Reshape, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
from sklearn.metrics import classification_report, confusion_matrix

In [18]:
# def build_model(input_shape=(224, 224, 3)):
#     base_model = VGG19(weights='imagenet', include_top=True, input_tensor=Input(shape=input_shape))
#     # Get the output of the 'fc2' layer in VGG16
#     a = base_model.get_layer('fc2').output

#     # Add a Dense layer with 13 neurons
#     dense_layer = Dense(13, activation='relu')(a)

#     # Add a final output layer with sigmoid activation
#     output_layer = Dense(1, activation='sigmoid')(dense_layer)

#     # Define the model with VGG16 base and the added layers
#     model = Model(inputs=base_model.input, outputs=output_layer)

#     # Freeze the weights of the VGG16 layers
#     for layer in base_model.layers:
#         layer.trainable = False

#     return model

def build_model(input_shape=(224, 224, 3), use_float16=False):
    inputs = Input(shape=input_shape)

    # Convert to lower precision if specified
    if use_float16:
        x = tf.keras.layers.Lambda(lambda t: tf.cast(t, tf.float16))(inputs)
    else:
        x = inputs

    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = BatchNormalization()(x)

    x = TimeDistributed(Flatten())(x)  # Flatten along the time dimension
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Attention()([x, x])  # Self-attention mechanism

    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)  # Adding dropout with a rate of 0.5
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)  # Adding dropout with a rate of 0.5
    
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)

    return model


In [19]:
  # base_model = VGG19(weights='imagenet', include_top=True,
  #                   input_tensor=Input(shape=(224, 224, 3)))
  # base_model.summary()

In [20]:
vgg_model = build_model()
vgg_model.summary()

2024-02-29 20:57:11.249850: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-29 20:57:11.277491: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-29 20:57:11.277917: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 224, 224, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, 224, 224, 64)         36928     ['conv2d[0][0]']              
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)         0         ['conv2d_1[0][0]']            
 D)                                                                                           

In [21]:
vgg_model.compile(
  optimizer=Adam(lr=0.01),
  loss='binary_crossentropy',
  metrics=['accuracy']
)

In [22]:
# batch_size = 16
# history = vgg_model.fit(
#     X_train,
#     y_train,
#     batch_size=batch_size,
#     epochs=35,
#     callbacks=[
#         ReduceLROnPlateau(
#             monitor = 'accuracy',
#             factor = 0.2,
#             patience = 5,
#             verbose = 1,
#             min_lr = 0.0001
#         ),
#         EarlyStopping(
#             monitor = 'accuracy',
#             patience = 10,
#             verbose = 1,
#             restore_best_weights = True
#         )
#     ]
# )

batch_size = 40
history = vgg_model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=60,
    callbacks=[
        ReduceLROnPlateau(
            monitor = 'accuracy',
            factor = 0.2,
            patience = 5,
            verbose = 1,
            min_lr = 0.0001
        ),
        # early_stop
    ]
)

Epoch 1/60


W0000 00:00:1709258236.929825   85774 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA GeForce RTX 3080" frequency: 1710 num_cores: 68 environment { key: "architecture" value: "8.6" } environment { key: "cuda" value: "12020" } environment { key: "cudnn" value: "8904" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 5242880 shared_memory_size_per_multiprocessor: 102400 memory_size: 7803568128 bandwidth: 760080000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
2024-02-29 20:57:17.342789: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-02-29 20:57:17.461313: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-29 20:57:20.147165: I external/local_xla/xla/service/serv

51/51 [==============================] - 28s 147ms/step - loss: 0.7825 - accuracy: 0.5926 - lr: 0.0010
Epoch 2/60
51/51 [==============================] - 6s 116ms/step - loss: 0.6926 - accuracy: 0.6321 - lr: 0.0010
Epoch 3/60
51/51 [==============================] - 6s 116ms/step - loss: 0.6614 - accuracy: 0.6430 - lr: 0.0010
Epoch 4/60
51/51 [==============================] - 6s 114ms/step - loss: 0.6475 - accuracy: 0.6625 - lr: 0.0010
Epoch 5/60
51/51 [==============================] - 6s 114ms/step - loss: 0.6124 - accuracy: 0.6750 - lr: 0.0010
Epoch 6/60
51/51 [==============================] - 6s 117ms/step - loss: 0.5979 - accuracy: 0.7109 - lr: 0.0010
Epoch 7/60
51/51 [==============================] - 6s 120ms/step - loss: 0.5571 - accuracy: 0.7204 - lr: 0.0010
Epoch 8/60
51/51 [==============================] - 6s 115ms/step - loss: 0.5122 - accuracy: 0.7698 - lr: 0.0010
Epoch 9/60
51/51 [==============================] - 6s 118ms/step - loss: 0.4936 - accuracy: 0.7659 - lr: 

In [23]:
# predictions
vgg_pred = vgg_model.predict(X_test, batch_size=1)

vgg_pred = np.round(vgg_pred)

  # model evaluation
confusion = confusion_matrix(y_test, vgg_pred)
print(classification_report(y_test, vgg_pred))
print(confusion)

W0000 00:00:1709258616.935448   85774 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA GeForce RTX 3080" frequency: 1710 num_cores: 68 environment { key: "architecture" value: "8.6" } environment { key: "cuda" value: "12020" } environment { key: "cudnn" value: "8904" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 5242880 shared_memory_size_per_multiprocessor: 102400 memory_size: 7803568128 bandwidth: 760080000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


501/501 [==============================] - 3s 3ms/step
              precision    recall  f1-score   support

           0       0.75      0.83      0.79       344
           1       0.52      0.39      0.44       157

    accuracy                           0.69       501
   macro avg       0.63      0.61      0.62       501
weighted avg       0.68      0.69      0.68       501

[[287  57]
 [ 96  61]]


In [24]:
vgg_model.save('./model_prolongation', overwrite=True)

INFO:tensorflow:Assets written to: ./model_prolongation/assets


INFO:tensorflow:Assets written to: ./model_prolongation/assets
